In [1]:
# Install necessary libraries
!pip install onnxruntime
!pip install insightface
!pip install opencv-python
!pip install pandas
!pip install scikit-learn
!pip install torch torchvision

# Import libraries
import os
import cv2
import numpy as np
import pandas as pd
import pickle
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached insightface-0.7.3.tar.gz (439 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached onnx-1.17.0-cp310-cp310-win_amd64.whl.metadata (16 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scikit_image-0.25.2-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached easydict-1.13-py3-none-any.whl.metadata (4.2 kB)
  Using cached Cython-3.0.12-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Using cached albumentations-2.0.6-py3-none-any.whl.metadata (43 kB)
  Using cached prettytable-3.16.0-py3-none-any.whl.metadata (33 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Load InsightFace FaceAnalysis model
def load_face_analyzer():
    app = FaceAnalysis(name="buffalo_l", providers=['CPUExecutionProvider'])
    app.prepare(ctx_id=0, det_size=(640, 640))
    return app

# Initialize model
face_app = load_face_analyzer()

# face_app now holds the face detection and recognition engine, ready to be used to analyze images.

print("✅ Model loaded successfully.")


download_path: C:\Users\dhruv/.insightface\models\buffalo_l


100%|██████████| 281857/281857 [00:09<00:00, 29607.40KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dhruv/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dhruv/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dhruv/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dhruv/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\dhruv/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [10]:
# import zipfile
# import os

# # upload dataset to colab
# from google.colab import files
# uploaded = files.upload()

# # unzip the dataset
# with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
#     zip_ref.extractall(".")

# print("✅ Dataset extracted!")

import zipfile
import os

# Path to your local zip file
zip_path = "dataset.zip"
extract_to = "."

# Unzip only if folder doesn't already exist
if not os.path.exists("dataset"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Dataset extracted!")
else:
    print("ℹ️ Dataset already extracted.")



✅ Dataset extracted!


In [11]:
# Function to build and empty dictionary face_db to store:
#Key: Folder name (e.g., 'ID001_John')
#Value: Average face embedding (512-dimensional NumPy array)

def build_database(dataset_path='dataset', database_file='face_database.pkl'):
    face_db = {}

    # Iterates through every subfolder name (i.e., person folders) in the dataset directory.
    for person_folder in os.listdir(dataset_path):

        # Constructs the full path to the person’s folder using os.path.join.
        person_path = os.path.join(dataset_path, person_folder)
        if os.path.isdir(person_path):  #Checks if the path is indeed a folder (not a file), to avoid errors.
            embeddings = []

            # Iterates through all image filenames inside that person’s folder.
            for img_file in os.listdir(person_path):
                img_path = os.path.join(person_path, img_file)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                faces = face_app.get(img)    # Runs face detection + embedding extraction on the image.
                if faces:
                    embeddings.append(faces[0].embedding)

            # Stores this avg embedding in the face_db dictionary with key = folder name ('ID001_John')
            if embeddings:
                avg_embedding = np.mean(embeddings, axis=0)
                face_db[person_folder] = avg_embedding

    # Save database
    with open('face_database.pkl', 'wb') as f:
        pickle.dump(face_db, f)

    print(f"✅ Database built with {len(face_db)} people.")

# Build it
build_database('dataset')


c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


✅ Database built with 5 people.


In [ ]:
import os
import cv2
import pickle
import numpy as np

# Step 1: Get ID and name
new_id = input("Enter new person's ID: ").strip()
new_name = input("Enter new person's Name: ").strip()
folder_name = f"{new_id}_{new_name}"
folder_path = os.path.join("dataset", folder_name)

# Step 2: Create directory for the new person
os.makedirs(folder_path, exist_ok=True)
print(f"✅ Created folder: {folder_path}")

# Step 3: Ask user to copy or specify image files
print("\n📂 Please place the new person's image files in this folder:")
print(f"   --> {folder_path}")
input("🛑 Press Enter after you've added the image(s) and are ready to continue...")

# Step 4: Collect image paths
image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

if not image_paths:
    print("❌ No images found in the folder.")
    exit()

# Step 5: Load existing face database
if os.path.exists('face_database.pkl'):
    with open('face_database.pkl', 'rb') as f:
        face_db = pickle.load(f)
else:
    face_db = {}

# Step 6: Generate embeddings using face_app
embeddings = []
for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Warning: Could not read image: {img_path}")
        continue
    faces = face_app.get(img)
    if faces:
        embeddings.append(faces[0].embedding)
    else:
        print(f"⚠️ No face detected in: {img_path}")

# Step 7: Add to database
if embeddings:
    avg_embedding = np.mean(embeddings, axis=0)
    face_db[folder_name] = avg_embedding
    print(f"✅ Embedding added for {folder_name}")
else:
    print("❌ No valid faces detected. Person not added.")
    exit()

# Step 8: Save updated database
with open('face_database.pkl', 'wb') as f:
    pickle.dump(face_db, f)

print("✅ face_database.pkl updated with the new person.")


✅ Created folder: dataset\_

📂 Please place the new person's image files in this folder:
   --> dataset\_
❌ No images found in the folder.
❌ No valid faces detected. Person not added.
✅ face_database.pkl updated with the new person.


: 

In [6]:
# Load group photo
group_photo_path = '/content/grp_photo5.png'
img = cv2.imread(group_photo_path)

# Runs face detection + embedding extraction on the image.
group_faces = face_app.get(img)

print(f"✅ Detected {len(group_faces)} faces in group photo.")


✅ Detected 6 faces in group photo.


In [7]:
# Load database
# Loads the dictionary face_db containing:
# Keys: person IDs like 'ID001_John'
# Values: 512-dimensional face embeddings (NumPy arrays)

with open('face_database.pkl', 'rb') as f:
    face_db = pickle.load(f)

known_ids = list(face_db.keys())  # list of folder names (IDs) from the database eg-['ID001_John']
known_embeddings = np.stack(list(face_db.values()))

# Match each detected face
attendance_records = []   # will store tuples like ('ID001_John', 'Present')
detected_ids = set()      # used to keep track of unique IDs already matched (helps avoid marking one person twice)

threshold = 0.7  # Cosine similarity threshold

print(f"Known IDs in database: {known_ids}")
print(f"Detected faces in group photo: {len(group_faces)}")

# Loops through each detected face (face) in the group image.
for i, face in enumerate(group_faces):
    embedding = face.embedding.reshape(1, -1)

    # Computes cosine similarity btw the detected face embedding and all known embeddings.
    similarities = cosine_similarity(embedding, known_embeddings)[0]

    # Finds the index of the highest similarity score, and extracts that score.
    best_match_idx = np.argmax(similarities)
    best_score = similarities[best_match_idx]

    print(f"\nFace #{i+1}")
    print(f"Best similarity score: {best_score}")

    if best_score > threshold:
        matched_id = known_ids[best_match_idx]
        print(f"✅ Matched with: {matched_id}")
        attendance_records.append((matched_id, "Present"))
        detected_ids.add(matched_id)
    else:
        print("❌ Unknown face detected")
        attendance_records.append(("Unknown", "Unknown"))


Known IDs in database: ['ID003_Ramos', 'ID005_Kroos', 'ID001_Courtois', 'ID002_Ronaldo', 'ID004_Benzema', 'ID008_Bale']
Detected faces in group photo: 6

Face #1
Best similarity score: 0.8646641969680786
✅ Matched with: ID002_Ronaldo

Face #2
Best similarity score: 0.8066542148590088
✅ Matched with: ID003_Ramos

Face #3
Best similarity score: 0.73204505443573
✅ Matched with: ID008_Bale

Face #4
Best similarity score: 0.7400767207145691
✅ Matched with: ID004_Benzema

Face #5
Best similarity score: 0.011730385944247246
❌ Unknown face detected

Face #6
Best similarity score: 0.04959612712264061
❌ Unknown face detected


In [8]:
# Mark absentees
for id_name in known_ids:
    if id_name not in detected_ids:
        attendance_records.append((id_name, "Absent"))

# Export CSV
df = pd.DataFrame(attendance_records, columns=["ID_Name", "Status"])
df[['ID', 'Name']] = df['ID_Name'].str.split('_', expand=True)
df = df[['ID', 'Name', 'Status']]
df.to_csv('attendance.csv', index=False)

print("✅ Attendance exported to attendance.csv successfully!")


✅ Attendance exported to attendance.csv successfully!
